# Retrieval Chain
> Source: [python.langchain.com/docs/get_started/quickstart/#retrieval-chain](https://python.langchain.com/docs/get_started/quickstart/#retrieval-chain)

To properly answer the question ("how can langsmith help with testing?"), we need to provide additional context to the LLM. We can do this via retrieval. Retrieval is useful when you have **too much data** to pass to the LLM directly. You can then use a retriever to fetch only the most relevant pieces and pass those in.

In this process, we will look up relevant documents from a _Retriever_ and then pass them into the prompt. A Retriever can be backed by anything - a SQL table, the internet, etc - but in this instance we will populate a vector store and use that as a retriever. For more information on vectorstores, see [this documentation](https://python.langchain.com/docs/modules/data_connection/vectorstores/).

First, we need to load the data that we want to index. To do this, we will use the WebBaseLoader. This requires the use of [BeautifulSoup](https://beautiful-soup-4.readthedocs.io/en/latest/):

In [16]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [17]:
print(docs)

[Document(page_content="\n\n\n\n\nLangSmith User Guide | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\nSkip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for 🦜️🛠️ LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.User GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly un

Next, we need to index it into a vectorstore. This requires a few components, namely an [embedding model](https://python.langchain.com/docs/modules/data_connection/text_embedding/) and a [vectorstore](https://python.langchain.com/docs/modules/data_connection/vectorstores/).

In [18]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

Now, we can use this embedding model to ingest documents into a vectorstore. We will use a simple local vectorstore, [FAISS](https://python.langchain.com/docs/integrations/vectorstores/faiss/), for simplicity's sake.

In [19]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vectorstore = FAISS.from_documents(documents, embeddings)

Now that we have this data indexed in a vectorstore, we will create a retrieval chain. This chain will take an incoming question, look up relevant documents, then pass those documents along with the original question into an LLM and ask it to answer the original question.

First, let's set up the chain that takes a question and the retrieved documents and generates an answer.

In [20]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

llm = ChatOpenAI()

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

We want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [21]:
from langchain.chains import create_retrieval_chain

retriever = vectorstore.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

We can now invoke this chain. This returns a dictionary - the response from the LLM is in the `answer` key

In [22]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

LangSmith can help with testing by allowing developers to create datasets, run tests on LLM applications, upload test cases in bulk, create custom evaluations, compare different versions of applications, use a playground environment for rapid iteration and experimentation, perform beta testing to collect real-world performance data, capture human feedback on responses, annotate traces, add runs to datasets, and closely inspect key data points in production. Additionally, LangSmith provides monitoring charts, A/B testing capabilities, automations for processing traces in near real-time, and threads view for tracking the performance of multi-turn applications.


# LCEL full example

In [24]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from textwrap import dedent

In [29]:
# get the documents
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()

# chunk and store them in the vectorstore
embeddings = OpenAIEmbeddings()
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vectorstore = FAISS.from_documents(documents, embeddings)

# create the retriever
retriever = vectorstore.as_retriever()

# create the model
llm = ChatOpenAI()

# create prompt template
template = dedent(
    '''\
    Answer the following question based only on the provided context:
        
    <context>
    {context}
    </context>

    Question: {input}
    '''
)

prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

# combine everything in a chain
chain = (
    # Do the following two things in parallel:
    # 1. Use the input to retrieve the relevant documents and assign them to the "context"
    # 2. Pass the input through to the prompt to re-use
    RunnableParallel(
        {"context": retriever, "input": RunnablePassthrough()}
    )
    | prompt
    | llm
    | output_parser
)

In [26]:
chain.invoke('how can langsmith help with testing?')

'LangSmith can help with testing by allowing developers to create datasets, run tests on LLM applications using these datasets, and easily run custom evaluations to score test results. It also provides a comparison view for tracking and diagnosing regressions in test scores across multiple revisions of an application. Additionally, LangSmith offers a playground environment for rapid iteration and experimentation, as well as the ability to log playground runs for creating test cases or comparing with other runs.'